# Code Setup

In [33]:
using Pkg; Pkg.activate(".")
using PyPlot
using MUST
using LaTeXStrings
using DelimitedFiles

MUST.@import_dispatch "/u/peitner/DISPATCH/dispatch2" ## Set the path of the current dispatch installation to find the models
MUST.@get_help visual;                                ## Import functions for plotting

  Activating environment at `/cobra/ptmp/peitner/model_grid/MUST.jl/examples/Paper_I/Project.toml`


# Mode Comparisons
We compare the models we obtain from DISPATCH with legacy models from Stagger, Co3bold and possibly other 1D models. For this compute the average structure of the 3D models on the geometrical and optical scale horizontally. 
We obtain different models using different EoS. We also use the Stagger opacities directly in DISPATCH to obtain a better comparison.

## Loading Models

Currently Tagged Models:

    1.) TSO_sun_v10.2_TDZ12: Matching profile to Stagger in the inerior, has a very hot surface
    2.) DIS_MARCS_v0.1.1_TDZ6: Reproduces the Stagger surface very well, large missmatch in the interiorß

In [34]:
## Stagger reference model (geometrical scale + optical depth scale (Rosseland))
folder_stagger = "/ptmp/peitner/model_grid/MUST.jl/examples/stagger2bifrost"
stagger        = MUST.Box("box_solar_stagger_LegacyStagger", folder=folder_stagger)
stagger_τ      = MUST.Box("box_solar_stagger_LegacyStagger_t", folder=folder_stagger);

In [35]:
## DISPATCH models (geometrical scale + optical depth scale (Rosseland))
### (A) Models with Turbospectrum opacities
folder_ts1             = MUST.@in_dispatch "data/TSO_sun_v10.2_TDZ4"
model_ts1, model_ts1_τ = MUST.Boxes(folder_ts1);

### (B) Models with AESOPUS 2.0 opacities
folder_ae1             = MUST.@in_dispatch "data/DIS_AESOPUS_v1.3.2_profile"
model_ae1, model_ae1_τ = MUST.Boxes(folder_ae1);

### (C) Models with Legacy opacities
folder_le1             = MUST.@in_dispatch "data/LEG_sun_test_TDZ_corr"
model_le1, model_le1_τ = MUST.Boxes(folder_le1);

## Setup and averaging

For the plotting we compare the horizontal averages and the differences

In [36]:
## Functions for keeping the figure layout homogeneous
default_2figure() = begin
    f, ax = plt.subplots(1, 2, figsize=(12,6))
    basic_plot!.(ax)

    f, ax
end

default_1figure() = begin
    f, ax = plt.subplots(1, 1, figsize=(9,6))
    basic_plot!(ax)

    f, ax
end;

## Use the averaging function from MUST package (StatsBase)
mean = MUST.mean;

We want to use a time statistic over the last snapshots for the comparison

In [37]:
time_r = 10
read_precomputed = false
snapshot_range(model, r=time_r)     = max(length(model) - r, 1):length(model)   ## Use the last snapshot_range snapshots
read_avsnap(folder::String, ext="") = MUST.Box("t$(time_r)$(ext).hdf5", folder=folder)
read_avsnap(models)                 = MUST.time_statistic(mean, view(model, snapshot_range(model)))

## (A)
#tmodel_ts1   = read_avsnap(folder_ts1)         
#tmodel_ts1_τ = read_avsnap(folder_ts1, "_tau") 
tmodel_ts1   = read_avsnap(model_ts1)
tmodel_ts1_τ = read_avsnap(model_ts1_τ)

## (B)
tmodel_ae1   = read_avsnap(model_ae1)
tmodel_ae1_τ = read_avsnap(model_ae1_τ)


## (C)
#tmodel_le1   = read_avsnap(folder_le1)         # read_avsnap(model_le1)
#tmodel_le1_τ = read_avsnap(folder_le1, "_tau") # read_avsnap(model_le1_τ);
tmodel_le1   = read_avsnap(model_le1)
tmodel_le1_τ = read_avsnap(model_le1_τ);

In [38]:
## Saving time averagers if needed for later
## (A)
MUST.save(tmodel_ts1,   name="t$(time_r).hdf5",     folder=folder_ts1)
MUST.save(tmodel_ts1_τ, name="t$(time_r)_tau.hdf5", folder=folder_ts1)

## (B) 
MUST.save(tmodel_ae1,   name="t$(time_r).hdf5",     folder=folder_ae1)
MUST.save(tmodel_ae1_τ, name="t$(time_r)_tau.hdf5", folder=folder_ae1)

## (C)
MUST.save(tmodel_le1,   name="t$(time_r).hdf5",     folder=folder_le1)
MUST.save(tmodel_le1_τ, name="t$(time_r)_tau.hdf5", folder=folder_le1);

ArgumentError: ArgumentError: invalid ASCII at index 1 in "τ_ross"

## Density - Temperature Comparison

In [39]:
close()
f, ax = default_1figure()


## The Dispatch models
begin
    ## (A)
    d = MUST.plane_statistic(mean, tmodel_ts1, :d)
    t = MUST.plane_statistic(mean, tmodel_ts1, :T)
    ax.plot(d, t, label="TS", color="r", lw=2)

    ## (B)
    d = MUST.plane_statistic(mean, tmodel_ae1, :d)
    t = MUST.plane_statistic(mean, tmodel_ae1, :T)
    ax.plot(d, t, label="TS", color="r", lw=2)

    ## (C)
    d = MUST.plane_statistic(mean, tmodel_le1, :d)
    t = MUST.plane_statistic(mean, tmodel_le1, :T)
    ax.plot(d, t, label="Legacy", color="b", lw=2)
end


## The Stagger model for comparison
d = MUST.plane_statistic(mean, stagger, :d)
t = MUST.plane_statistic(mean, stagger, :T)
ax.plot(d, t, label="Stagger", color="k", lw=2.5)


## Plot setup
ax.set_xscale("log")

gcf()
